# Using free and open source tools to analyze data from the Federal Trade Commission (FTC)

This notebook shows how to use free open source tools -- Python and Tableau Public -- to analyze Do Not Call program data made available to the public by the Federal Trade Commission.   

The data used in the analysis below was taken from https://www.ftc.gov/site-information/open-government/data-sets/do-not-call-data.  It includes Do Not Call and robocall reports to the Federal Trade Commission. The data contains information reported by consumers, including the telephone number originating the unwanted call, the date the complaint was created, the time the call was made, the consumer’s city and state locations reported, the subject of the call, the consumers area code and whether the call was a robocall. 


We will use python to automatically pull data from the web, clean it, and create a data set that can be used to build interactive dashboards with Tableau.  

The dashboards are made available on Tableau Public – a free service that allows users to publish dashboards to the 
web.

The dashboards are located at: https://public.tableau.com/profile/paul.witt2290#!/  or through NBViewer with this notebook.
 

## Secton I: Data Wrangling with Python

### Requests and Beautiful Soup

We start by using the requests python library to access the HTML code from FTC.gov.  http://docs.python-requests.org/en/master/

The requests library has a straight forward API that allows us to easliy request data from FTC.gov. 


Below we create a response object to retrive the web page that contains the data we need. The response object contains a server’s response to an HTTP request. The .get method below initiates an HTTP Get request. 

For more on HTTP requests see https://www.w3schools.com/tags/ref_httpmethods.asp


In [2]:
import requests  

r = requests.get('https://www.ftc.gov/site-information/open-government/data-sets/do-not-call-data') 

We now have a response object that gives us access to the elements of Document Oject Model and allows us to inspect the webpage that contains our data.  

Here is a small sample of the output. 

In [70]:
print(r.text[28500:29500]) 

="header-wrapper">

  <header id="header" role="banner">

    
              
          <div class="logo-bg">
            <a href="/" title="Home" rel="home" id="logo">
                                             <span data-picture data-alt="Federal Trade Commission: Protecting America's Consumers">
                <span data-src="/sites/all/themes/ftc/images/FTCLogo_465-new.png"></span>
                <span data-media="(max-width: 766px)" data-src="/sites/all/themes/ftc/images/ftc_logo_430.png"></span>
                <span data-media="(min-device-width: 766px) and (max-device-width: 1800px)" data-src="/sites/all/themes/ftc/images/FTCLogo_465-new.png"></span>
                <span data-media="(-webkit-min-device-pixel-ratio: 2), (min-resolution: 192dpi)" data-src="/sites/all/themes/ftc/images/FTCLogo_930-new.png"></span>
                 <span data-media="(max-width: 959px) and (-webkit-min-device-pixel-ratio: 2), (max-width: 959px) and (min-resolution: 192dpi)" data-src="/sites/all

We only need to access csv files, so most of what we see in the small sample above is not useful. We could use string operations to search and find what we need but that would be combersome and time consuming.  Instead, we will use the Beautiful Soup python library. The Beautiful Soup API will help us quickly parse the strings on this page to get at what we need. 

Beautiful Soup transforms a complex HTML document into a complex tree of Python objects. For our purposes, the ability to quickly seach and access the tags, attributes and elements in the webpage will be necessary to retrieve our data. 

For more on HTML objects see: https://www.456bereastreet.com/archive/200508/html_tags_vs_elements_vs_attributes/

We start by importing Beautiful Soup Library. We will pass our response object into a BS HTML parser. 


In [4]:
from bs4 import BeautifulSoup  
soup = BeautifulSoup(r.text, 'html.parser') 

We now have a parsed oject that we can apply simple Beautiful Soup methods to.

Our goal is to find all of the csv files on the webpage, which are hyperlinks.  As such, we want to isolate all the '< a >' tags and 'href' attributes.   

The '< a >' tag defines a hyper link in a webpage. For more on < a > tags see: https://www.w3schools.com/tags/tag_a.asp

The href attribute specifies the link's destination: https://www.w3schools.com/tags/att_a_href.asp

We can use the Beautiful Soup find_all method to grab all the links in the webpage.   From there, we will use a python list comprehension to find and isolate all the csv files we need in the web page. Because the naming convention is clean, we further isolate the specific csv files we are looking for by only returning the links we need.   



In [54]:
def get_links():
    
    links = soup.find_all('a',href=True)
    
    links = [link['href'] for link in links
        if link["href"].startswith\
        ("https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_")]

    return links

get_links()


[u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-26.csv',
 u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-19.csv',
 u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-12.csv',
 u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-25.csv',
 u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-18.csv',
 u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-11.csv',
 u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-24.csv',
 u'https://www.ftc.gov/system/files/attachments/do-not-call-dnc-reported-calls-data/dnc_complaint_numbers_2018-10-17.csv',
 u'https://www.f

## Data Exploration with Pandas

Now that we have a list of links to our csv files, we will use the pandas library to put it all together. https://pandas.pydata.org/pandas-docs/stable/

We start by using another list comprehension to read each csv link. The end result is a new list that contains one dataframe for each hyperlink. We then append all the dataframes together into one final dataframe that we can use for analysis. 

In [56]:
import pandas as pd

def read_links(data):
    dfs = [pd.read_csv(link,error_bad_lines=False) for link in data]
    
    df = pd.concat(dfs, ignore_index=True)
    
    return df

dnc_data = read_links(get_links())
    
    

Skipping line 11650: expected 8 fields, saw 15



In [57]:
dnc_data

,Company_Phone_Number,Created_Date,Violation_Date,Consumer_City,Consumer_State,Consumer_Area_Code,Subject,Recorded_Message_Or_Robocall
0,6.50238e+09,2018-10-25 00:00:34,2018-10-24 17:50:00,Westwood,New Jersey,201.0,Medical & prescriptions,N
1,6.50204e+09,2018-10-25 00:00:36,2018-10-24 20:50:00,Belmont,California,650.0,No Subject Provided,Y
2,5.05492e+09,2018-10-25 00:00:46,2018-10-22 19:54:00,Albuquerque,New Mexico,505.0,Dropped call or no message,N
3,7.60657e+09,2018-10-25 00:01:25,2018-10-24 17:37:00,NaN,New Mexico,505.0,"Reducing your debt (credit cards, mortgage, st...",N
4,2.01878e+09,2018-10-25 00:02:06,2018-10-24 18:11:00,Westwood,New Jersey,201.0,Medical & prescriptions,N
5,6.50681e+09,2018-10-25 00:03:00,2018-10-24 18:09:00,Los Altos,California,408.0,Dropped call or no message,N
6,4.08385e+09,2018-10-25 00:04:08,2018-10-24 17:44:00,San Francisco,California,415.0,No Subject Provided,Y
7,5.51581e+09,2018-10-25 00:04:19,2018-10-24 14:00:00,NaN,New Jersey,908.0,Medical & prescriptions,Y
8,5.09223e+09,2018-10-25 00:05:15,2018-10-24 20:40:00,NaN,Washington,509.0,Dropped call or no message,Y
9,6.82777e+09,2018-10-25 00:05:49,2018-10-24 12:52:00,San Francisco,California,415.0,"Calls pretending to be government, businesses,...",Y


In [71]:
dnc_data.count()

Company_Phone_Number            695786
Created_Date                    717774
Violation_Date                  717773
Consumer_City                   452521
Consumer_State                  717183
Consumer_Area_Code              717763
Subject                         717773
Recorded_Message_Or_Robocall    709752
dtype: int64

In [74]:
dnc_data=dnc_data[dnc_data.Created_Date!='N']

In [75]:
dnc_data.Created_Date=pd.to_datetime(dnc_data.Created_Date)

/Users/pwitt/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [76]:
dnc_data.Created_Date.dt.day.unique()

array([25, 18, 11, 24, 17, 10, 23, 16,  9, 22, 15,  8, 19, 20, 21, 12, 13,
        5,  6,  7,  4, 27,  3, 26,  2,  1, 28, 29, 30, 14])

In [77]:
dnc_data.Created_Date.max()

Timestamp('2018-10-25 23:59:45')

In [78]:
dnc_data.Created_Date.min()

Timestamp('2018-09-14 00:00:07')

In [79]:
dnc_data.Company_Phone_Number=dnc_data.Company_Phone_Number.astype(str)

Nice Clean data set ready for Tableau. 

In [80]:
dnc_data.to_csv('data/dnc_pull.csv')